In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from plotly.offline import init_notebook_mode, iplot, plot
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM, Dense, SpatialDropout1D, Dropout
import h5py
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
ds= pd.read_csv('/content/drive/MyDrive/code/dataset/df_spark.csv')
ds.shape
ds.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11
0,0,7.0,21.0,30.0,2.0,3.0,30.0,2.0,3.0,59.0,7.0,2.0,0.0
1,1,7.0,32.0,70.0,2.0,4.0,68.0,2.0,4.0,134.0,7.0,2.0,0.0
2,2,7.0,15.0,1.0,2.0,2.0,0.0,2.0,2.0,0.0,7.0,2.0,0.0
3,3,7.0,33.0,77.0,2.0,7.0,73.0,2.0,7.0,144.0,7.0,2.0,0.0
4,4,7.0,55.0,78.0,3.0,7.0,74.0,3.0,7.0,146.0,8.0,2.0,0.0


In [ ]:
ds = ds.drop(columns="Unnamed: 0")
ds.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.0,21.0,30.0,2.0,3.0,30.0,2.0,3.0,59.0,7.0,2.0,0.0
1,7.0,32.0,70.0,2.0,4.0,68.0,2.0,4.0,134.0,7.0,2.0,0.0
2,7.0,15.0,1.0,2.0,2.0,0.0,2.0,2.0,0.0,7.0,2.0,0.0
3,7.0,33.0,77.0,2.0,7.0,73.0,2.0,7.0,144.0,7.0,2.0,0.0
4,7.0,55.0,78.0,3.0,7.0,74.0,3.0,7.0,146.0,8.0,2.0,0.0


In [ ]:
y = ds.iloc[:,0].values
X = ds.iloc[:,1:].values

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

from sklearn.preprocessing import label_binarize
y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7])
n_classes = y.shape[1]
print(n_classes)

8


In [ ]:
#With CV
for train, test in kfold.split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))

    model = Sequential()
    model.add(Bidirectional(LSTM(80, input_dim=11, return_sequences=True)))
    model.add(Dropout(0.1))
    model.add(Bidirectional(LSTM(40)))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(8, activation='softmax'))

    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy','mae'])
    model.fit(X_train, y_train, epochs=15, batch_size=64, verbose=1)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0521 - accuracy: 0.9882 - mae: 0.0061
Epoch 2/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0152 - accuracy: 0.9933 - mae: 0.0024
Epoch 3/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0132 - accuracy: 0.9939 - mae: 0.0021
Epoch 4/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0130 - accuracy: 0.9940 - mae: 0.0021
Epoch 5/15
4475/4475 [==============================] - 31s 7ms/step - loss: 0.0123 - accuracy: 0.9942 - mae: 0.0020
Epoch 6/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0130 - accuracy: 0.9941 - mae: 0.0021
Epoch 7/15
4475/4475 [==============================] - 35s 8ms/step - loss: 0.0123 - accuracy: 0.9942 - mae: 0.0020
Epoch 8/15
4475/4475 [==============================] - 30s 7ms/step - loss: 0.0124 - accuracy: 0.9942 - mae: 0.0020
Epoch 9/15
4475/4475 [==============================] - 32s 7ms/

In [ ]:
target_names = ['DoS', 'Probing',	'MaliciousControl', 'MaliciousOperation', 'Scan', 'Spying', 'WrongSetup', 'Normal']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

                    precision    recall  f1-score   support

               DoS       0.98      0.66      0.79      4622
           Probing       1.00      1.00      1.00       257
  MaliciousControl       1.00      1.00      1.00       706
MaliciousOperation       1.00      1.00      1.00       645
              Scan       1.00      1.00      1.00      1249
            Spying       1.00      1.00      1.00       409
        WrongSetup       1.00      1.00      1.00       100
            Normal       0.99      1.00      1.00    278364

         micro avg       0.99      0.99      0.99    286352
         macro avg       1.00      0.96      0.97    286352
      weighted avg       0.99      0.99      0.99    286352
       samples avg       0.99      0.99      0.99    286352

Accuracy_Train: 0.9942867519696038
Precision_Train: 0.9942158583153095
Recall_Train: 0.9942867519696038
f1-score_Train: 0.9937332519609938
ROC_AUC_Train: 0.9987677633988182


In [ ]:
### test_accuracy
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

                    precision    recall  f1-score   support

               DoS       0.99      0.63      0.77      1158
           Probing       0.99      1.00      0.99        85
  MaliciousControl       1.00      1.00      1.00       183
MaliciousOperation       1.00      1.00      1.00       160
              Scan       1.00      1.00      1.00       298
            Spying       1.00      1.00      1.00       123
        WrongSetup       1.00      1.00      1.00        22
            Normal       0.99      1.00      1.00     69560

         micro avg       0.99      0.99      0.99     71589
         macro avg       1.00      0.95      0.97     71589
      weighted avg       0.99      0.99      0.99     71589
       samples avg       0.99      0.99      0.99     71589

Accuracy: 0.9939236474877425
Precision: 0.9938736566305009
Recall: 0.9939236474877425
f1-score: 0.9932767283480071
ROC_AUC: 0.9986812807533668
